In [ ]:
from __future__ import division, print_function

import safe_rl
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

### Test the triangulation

Create a triangulation on a grid and plot the corresponding results.
Then take a test point and return in which simplex the corresponding point lies

The naming and indexing convention follows the standard C-indexing style (last index moves the fastest).

The space is split up into regular hyperrectangles, as given by the limits and npoints arguments. These rectangles are indexed from 0 onwards. Within each hyperrectangle is a delaunay triangulation, which is computed only for one of them (the code is basically a wrapper around ```scipy.spatial.Delaunay```.).

Within each hyperrectangle, the order of the triangle indeces is determined by the Delaunay decomposition. Outside, it is given by ```#rectangle * nsimplex + #simplex```, where ```#rectangle``` is the index of the rectangle, ```nsimplex``` is the number of simpleces per hyperrectangle, and ```#simplex``` is the index of the simplex within the rectangle, as given by the Delaunay decomposition.

In [ ]:
limits = [[-1, 1], [-1, 1]]
npoints = [2, 3]

# Create triangulation
tri = safe_rl.Delaunay(limits, npoints)

In [ ]:
# Get all simplices and points
simplices = tri.simplices(np.arange(tri.nsimplex))
points = tri.index_to_state(np.arange(tri.nindex))

# colors
simplex_color = 'g'
node_color = 'b'
rect_color = 'k'

# Plot nodes
plt.plot(points[:, 0], points[:, 1], 'o', color=node_color)

# Annotate nodes
for p in points:
    plt.text(p[0] - 0.03, p[1] + 0.03,
             tri.state_to_index(p)[0],
             ha='right', color=node_color)

# Annotate rectangles
rectangles = tri.rectangle_to_state(np.arange(tri.nrectangles))
rectangles += tri.unit_maxes / 2
for i, point in enumerate(rectangles):
    plt.text(point[0], point[1], '#%d' % i,
             ha='center', color=rect_color)

# Plot triangles
plt.triplot(points[:, 0], points[:, 1], simplices.copy(),
            color=simplex_color)

# Annotate triangles
for i, simplex in enumerate(simplices):
    p = points[simplex].mean(axis=0)
    plt.text(p[0], p[1], '#%d' % i,
             ha='center', color=simplex_color)

# Format plot nicely
offset = 0.1 * np.array([-1, 1])
plt.xlim(tri.limits[0] + offset * np.diff(tri.limits[0]))
plt.ylim(tri.limits[1] + offset * np.diff(tri.limits[1]))


# test points
test_point = np.array([[0.9, 0.9],
                       [-0.5, -0.2]])
plt.plot(test_point[0], test_point[1], 'x')
plt.show()

# Output the result of the evaluation
simplices = tri.find_simplex(test_point)
simplex_nodes = tri.simplices(simplices)

print('Evaluation results for test points:\n')
for index, nodes in zip(simplices, simplex_nodes):
    print("simplex {id} with nodes {nodes}".format(id=index, nodes=nodes))


### Naive implementation in scipy

This is just the vanilla scipy implementation of Delaunay

In [ ]:
tri = safe_rl.ScipyDelaunay(limits, npoints)
points = tri.points
simplices = tri.simplices

# Plot nodes
plt.plot(points[:, 0], points[:, 1], 'o', color=node_color)

# Annotate nodes
for i, p in enumerate(points):
    plt.text(p[0] - 0.03, p[1] + 0.03, i,
             ha='right', color=node_color)
    
# Plot triangles
plt.triplot(points[:, 0], points[:, 1], simplices.copy(),
            color=simplex_color)

# Annotate triangles
for i, simplex in enumerate(simplices):
    p = points[simplex].mean(axis=0)
    plt.text(p[0], p[1], '#%d' % i,
             ha='center', color=simplex_color)
    
# Format plot nicely
offset = 0.1 * np.array([-1, 1])
plt.xlim(tri.limits[0] + offset * np.diff(tri.limits[0]))
plt.ylim(tri.limits[1] + offset * np.diff(tri.limits[1]))

### Speed comparison

In [ ]:
test_points = np.arange(10, 51, 5)
ntrials = 5
time = np.empty((len(test_points), 2), dtype=np.float)

for i, n in enumerate(test_points):
    npoints = [n, n]

    time[i, 0] = timeit.timeit('import safe_rl; safe_rl.Delaunay({0}, {1})'
                             .format(limits, npoints),
                             number=ntrials)

    time[i, 1] = timeit.timeit('import safe_rl; safe_rl.ScipyDelaunay({0}, {1})'
                               .format(limits, npoints),
                               number=ntrials)

ntriangles = 2 * test_points ** 2

plt.plot(ntriangles, time[:, 0], label='efficient')
plt.plot(ntriangles, time[:, 1], label='scipy')
plt.ylabel('Computation time [s]')
plt.xlabel('Number of triangles')
plt.legend()
plt.show()